In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
import sys
sys.path.append('../')
import helper as hlp

## Fetch User Data

In [11]:
from datetime import datetime



ratings_db = hlp.mongoDBHelper('aieng', 'AB_ratings')

In [17]:
start_date = datetime.strptime('2022-04-10','%Y-%m-%d')
end_date = datetime.strptime('2022-04-15','%Y-%m-%d')

print(f"start_date: {start_date} end_date: {end_date}")

ratings_records = list(ratings_db.collection.find({'ts':{'$gte':start_date, '$lt':end_date},
                                                    'online_eval': {'$in':['hit','miss']}
}))
len(ratings_records)


start_date: 2022-04-10 00:00:00 end_date: 2022-04-15 00:00:00


86477

In [18]:
ratings_df = pd.DataFrame(ratings_records)
ratings_df.head()

,_id,userID,movieID,rating,ts,online_eval,modelID
0,62574c9e729efcdc10c0036a,971370,the+wrong+trousers+1993,4,2022-04-13 18:16:43.000,hit,KPSModelNotFound
1,62574f3d729efcdc10c01885,907805,one+flew+over+the+cuckoos+nest+1975,4,2022-04-13 18:31:25.369,hit,KPSModelNotFound
2,625750f7729efcdc10c026f1,55121,casablanca+1942,4,2022-04-13 18:21:54.000,hit,KPSModelNotFound
3,6257511b729efcdc10c0280b,806596,vesku+2010,3,2022-04-13 18:22:23.000,miss,KPSModelNotFound
4,62575185729efcdc10c02b9c,487236,no+and+me+2010,4,2022-04-13 18:22:53.000,miss,KPSModelNotFound


## Fetch user information

In [22]:
USERS_BASE_URL = "http://128.237.79.25:8080/user/"
def get_user_info(user_id):
    url = USERS_BASE_URL + str(user_id)
    r = requests.get(url)
    if r.ok:
        return r.json()
    else:
        return None

ratings_df['gender'] = None
ratings_df['age'] = -1

In [23]:
def get_info(row):
    user_id = row['userID']
    info = get_user_info(user_id)
    if info is not None:
        row['gender'] = info['gender']
        row['age'] = int(info['age'])

    return row

In [36]:
from tqdm import tqdm
tqdm.pandas()

ratings_df = ratings_df.progress_apply(lambda row: get_info(row), axis=1)

100%|██████████| 81974/81974 [17:45<00:00, 76.94it/s]   


In [27]:
display(ratings_df.info())
ratings_df = ratings_df.drop_duplicates(subset=['userID'])
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86477 entries, 0 to 86476
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   _id          86477 non-null  object        
 1   userID       86477 non-null  object        
 2   movieID      86477 non-null  object        
 3   rating       86477 non-null  object        
 4   ts           86477 non-null  datetime64[ns]
 5   online_eval  86477 non-null  object        
 6   modelID      86477 non-null  object        
 7   gender       0 non-null      object        
 8   age          86477 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 5.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81974 entries, 0 to 86475
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   _id          81974 non-null  object        
 1   userID       81974 non-null  object        
 2   movieID      81974 non-null  object        
 3   rating       81974 non-null  object        
 4   ts           81974 non-null  datetime64[ns]
 5   online_eval  81974 non-null  object        
 6   modelID      81974 non-null  object        
 7   gender       0 non-null      object        
 8   age          81974 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 6.3+ MB


In [ ]:
# updated_ratings = pd.DataFrame(columns=ratings_df.columns)

# for idx, row in tqdm(ratings_df.iterrows()):
#     row = get_info(row)
#     if row['gender'] is not None:
#         updated_ratings = updated_ratings.append(row)

In [40]:
ratings_df.describe(include='all')

/var/folders/8m/tt40_c8n0n18l8g1hyh4lj4h0000gn/T/ipykernel_18361/3204085927.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  ratings_df.describe(include='all')


,_id,userID,movieID,rating,ts,online_eval,modelID,gender,age
count,81974,81974,81974,81974,81974,81974,81974,81974,81974.000000
unique,81974,81974,15533,5,15856,2,1,2,NaN
top,62574c9e729efcdc10c0036a,971370,the+wrong+trousers+1993,4,2022-04-14 01:36:41,hit,KPSModelNotFound,M,NaN
freq,1,1,15737,42085,20,46252,81974,68213,NaN
first,NaN,NaN,NaN,NaN,2022-04-13 18:16:43,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-04-14 23:57:25.627000,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.078086
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.194095
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.000000


In [41]:
ratings_df.gender.value_counts()

M    68213
F    13761
Name: gender, dtype: int64

In [42]:
ratings_df['age_bucket'] = np.where(ratings_df.age.values > 35, "old", "young")
ratings_df

,_id,userID,movieID,rating,ts,online_eval,modelID,gender,age,age_bucket
0,62574c9e729efcdc10c0036a,971370,the+wrong+trousers+1993,4,2022-04-13 18:16:43.000,hit,KPSModelNotFound,M,78,old
1,62574f3d729efcdc10c01885,907805,one+flew+over+the+cuckoos+nest+1975,4,2022-04-13 18:31:25.369,hit,KPSModelNotFound,M,30,young
2,625750f7729efcdc10c026f1,55121,casablanca+1942,4,2022-04-13 18:21:54.000,hit,KPSModelNotFound,M,27,young
3,6257511b729efcdc10c0280b,806596,vesku+2010,3,2022-04-13 18:22:23.000,miss,KPSModelNotFound,F,32,young
4,62575185729efcdc10c02b9c,487236,no+and+me+2010,4,2022-04-13 18:22:53.000,miss,KPSModelNotFound,F,31,young
...,...,...,...,...,...,...,...,...,...,...
86469,6259029712f71b05c14a5a91,861797,why+we+fight+2005,5,2022-04-14 03:06:29.000,miss,KPSModelNotFound,M,32,young
86470,6259029712f71b05c14a5a92,452246,one+flew+over+the+cuckoos+nest+1975,4,2022-04-14 03:06:29.000,hit,KPSModelNotFound,M,30,young
86471,6259029712f71b05c14a5a93,478381,the+lion+king+2+simbas+pride+1998,2,2022-04-14 03:06:29.000,miss,KPSModelNotFound,F,41,old
86473,6259029812f71b05c14a5a95,161101,alaskaland+2013,3,2022-04-14 03:06:29.000,hit,KPSModelNotFound,M,26,young


In [44]:
online_eval_mapping = {
    'hit': 1,
    'miss': 0
}
ratings_df['eval_score'] = ratings_df.online_eval.map(online_eval_mapping)
ratings_df


,_id,userID,movieID,rating,ts,online_eval,modelID,gender,age,age_bucket,eval_score
0,62574c9e729efcdc10c0036a,971370,the+wrong+trousers+1993,4,2022-04-13 18:16:43.000,hit,KPSModelNotFound,M,78,old,1
1,62574f3d729efcdc10c01885,907805,one+flew+over+the+cuckoos+nest+1975,4,2022-04-13 18:31:25.369,hit,KPSModelNotFound,M,30,young,1
2,625750f7729efcdc10c026f1,55121,casablanca+1942,4,2022-04-13 18:21:54.000,hit,KPSModelNotFound,M,27,young,1
3,6257511b729efcdc10c0280b,806596,vesku+2010,3,2022-04-13 18:22:23.000,miss,KPSModelNotFound,F,32,young,0
4,62575185729efcdc10c02b9c,487236,no+and+me+2010,4,2022-04-13 18:22:53.000,miss,KPSModelNotFound,F,31,young,0
...,...,...,...,...,...,...,...,...,...,...,...
86469,6259029712f71b05c14a5a91,861797,why+we+fight+2005,5,2022-04-14 03:06:29.000,miss,KPSModelNotFound,M,32,young,0
86470,6259029712f71b05c14a5a92,452246,one+flew+over+the+cuckoos+nest+1975,4,2022-04-14 03:06:29.000,hit,KPSModelNotFound,M,30,young,1
86471,6259029712f71b05c14a5a93,478381,the+lion+king+2+simbas+pride+1998,2,2022-04-14 03:06:29.000,miss,KPSModelNotFound,F,41,old,0
86473,6259029812f71b05c14a5a95,161101,alaskaland+2013,3,2022-04-14 03:06:29.000,hit,KPSModelNotFound,M,26,young,1


In [47]:
male_df = ratings_df[ratings_df.gender == 'M'].copy()
female_df = ratings_df[ratings_df.gender == 'F'].copy()

display(male_df.online_eval.value_counts())
display(female_df.online_eval.value_counts())

hit     38386
miss    29827
Name: online_eval, dtype: int64

hit     7866
miss    5895
Name: online_eval, dtype: int64

In [48]:
def get_hit_ratio(df):
    hits = df['eval_score'].sum()
    return hits / len(df)

male_hit_ratio = get_hit_ratio(male_df)
female_hit_ratio = get_hit_ratio(female_df)

print(f"Male hit ratio: {male_hit_ratio}")
print(f"Female hit ratio: {female_hit_ratio}")

Male hit ratio: 0.562737308137745
Female hit ratio: 0.5716154349247874


In [52]:
young_df = ratings_df[ratings_df.age_bucket == 'young'].copy()
old_df = ratings_df[ratings_df.age_bucket == 'old'].copy()

young_hit_ratio = get_hit_ratio(young_df)
old_hit_ratio = get_hit_ratio(old_df)

print(f"Young adult hit ratio: {young_hit_ratio}")
print(f"Old adult hit ratio: {old_hit_ratio}")

Young adult hit ratio: 0.5634243793803628
Old adult hit ratio: 0.5751019684341195


In [50]:
young_df.online_eval.value_counts()

hit     43009
miss    33326
Name: online_eval, dtype: int64

In [51]:
old_df.online_eval.value_counts()

hit     3243
miss    2396
Name: online_eval, dtype: int64